In [ ]:
import pandas as pd
import os

In [ ]:
# working directory
working_dir = "/Users/augustus/Desktop/WSWC/WaDE/MappingStatesDataToWaDE2.0/Wyoming/AggregatedAmounts/RawInputData/"
os.chdir(working_dir)


In [ ]:
target_columns = ["OrganizationUUID", "VariableSpecificUUID", 
                  "ReportingUnitUUID",
                  "PrimaryUseCategory", "BeneficialUseCategory", 
                  "WaterSourceUUID", "MethodUUID", "TimeframeStart", "TimeframeEnd", 
                  "DataPublicationDate", "DataPublicationDOI", "ReportYearCV", "Amount",  
                  "PopulationServed", "PowerGeneratedGWh", "IrrigatedAcreage",
                  "InterbasinTransferToID", "InterbasinTransferFromID", "CustomerTypeCV",
                  "AllocationCropDutyAmount", "IrrigationMethodCV", "CropTypeCV",
                  "CommunityWaterSupplySystem", "SDWISIdentifierCV"]


In [ ]:

outdf100 = pd.DataFrame(columns=target_columns)

In [ ]:
#Input files
fileInput1 = "Agricultural, Municipal, Domestic, and Industrial Use Indecies_Draft_For Delivery to WWDC_2019.0703.xlsx" 


# reporting units lookup
inp_repunts = "reportingunits.csv"

# outputs aggregated amounts
out_agamount = "aggregatedamounts.csv"


In [ ]:
# only need county names in reporting units
# remove redundancies
df = df.drop_duplicates(subset='Water Use by Basin')
df = df.reset_index(drop=True)
outdf["ReportingUnitName"] = df["Water Use by Basin"]